In [1]:
import xarray as xr
import numpy as np
import time as t

#### Local time versus UTC

In [10]:
filtered_edges = np.load( 'CT-allDataFilteredEdges_tropical.npy' )
local_time = np.load( 'CT-allDataLocalTimes_tropical.npy' )

print( filtered_edges.shape )
print( local_time.shape )
print( '~~~~~~~~~~~~~~' )
times = filtered_edges[:,10]
not_multiples_of_three = times[ times%3 != 0 ]
print( not_multiples_of_three )

(803075, 49)
(803075, 49)
~~~~~~~~~~~~~~
[]


#### This cell copies fields from ERA-Interim files to ERA-5 files.

In [ ]:
var = [ 'z', 'lifetime' ]
for i in np.arange( 1983, 1984):#2009 ):
    print( i )
    file1 = xr.open_dataset( '/groups/sylvia/JAS-MCS-rain/ERAI/colloc_' + str(i) + '.nc' )
    file2 = xr.open_dataset( '/groups/sylvia/JAS-MCS-rain/ERA5/colloc5_' + str(i) + '.nc' )
    for v in var:
        file2[v] = file1[v]
    print(file2)

#### This cell extracts only the ISCCP data for a tropical domain.

In [3]:
hilat = 10
lowlat = -10
#ISCCP_data = np.load( "CT-allDataLocalTimes.npy" )
ISCCP_data = np.load( "CT-allDataFilteredEdges.npy" )
CT_lat = ISCCP_data[:,12]
print( CT_lat.min(), CT_lat.max() )

filtered_CT_data = ISCCP_data[(CT_lat >= lowlat) & (CT_lat <= hilat)]
#np.save( "CT-allDataLocalTimes_tropical.npy", filtered_CT_data )
np.save( "CT-allDataFilteredEdges_tropical.npy", filtered_CT_data )
print( filtered_CT_data[:,12].min(), filtered_CT_data[:,12].max() )

-51.0 51.9
-10.0 10.0


#### Testing / debugging capeCollocate_ERA5.py

In [2]:
filtered_data = np.load( "CT-allDataLocalTimes_tropical.npy" )
years = filtered_data[:,7]
data_1983 = filtered_data[(years == 1983.)]
print( data_1983[:,8] )

[ 8.  8.  8. ... 12. 12. 12.]


In [2]:
era_data = xr.open_dataset( '/xdisk/sylvia/ERA5_output/ERA5_qv_tropical.nc' )
#era_data = xr.open_dataset( '/xdisk/sylvia/ERA5_output/ERA5_cape_tropical.nc' )
#era_data = era_data.assign_coords( longitude=((era_data['longitude'] + 360) % 360) )

time = era_data['valid_time'].dt.round("S")
longitude = era_data['longitude']
display( time )

<xarray.DataArray 'round' (valid_time: 744)>
array(['1983-08-01T00:00:00.000000000', '1983-08-01T01:00:00.000000000',
       '1983-08-01T02:00:00.000000000', '1983-08-01T03:00:00.000000000',
       '1983-08-01T04:00:00.000000000', '1983-08-01T05:00:00.000000000',
       '1983-08-01T06:00:00.000000000', '1983-08-01T07:00:00.000000000',
       '1983-08-01T08:00:00.000000000', '1983-08-01T09:00:00.000000000',
       '1983-08-01T10:00:00.000000000', '1983-08-01T11:00:00.000000000',
       '1983-08-01T12:00:00.000000000', '1983-08-01T13:00:00.000000000',
       '1983-08-01T14:00:00.000000000', '1983-08-01T15:00:00.000000000',
       '1983-08-01T16:00:00.000000000', '1983-08-01T17:00:00.000000000',
       '1983-08-01T18:00:00.000000000', '1983-08-01T19:00:00.000000000',
       '1983-08-01T20:00:00.000000000', '1983-08-01T21:00:00.000000000',
       '1983-08-01T22:00:00.000000000', '1983-08-01T23:00:00.000000000',
       '1983-08-02T00:00:00.000000000', '1983-08-02T01:00:00.000000000',
       '1983-08-02T02:00:00.000000000', '1983-08-02T03:00:00.000000000',
       '1983-08-02T04:00:00.000000000', '1983-08-02T05:00:00.000000000',
       '1983-08-02T06:00:00.000000000', '1983-08-02T07:00:00.000000000',
       '1983-08-02T08:00:00.000000000', '1983-08-02T09:00:00.000000000',
       '1983-08-02T10:00:00.000000000', '1983-08-02T11:00:00.000000000',
       '1983-08-02T12:00:00.000000000', '1983-08-02T13:00:00.000000000',
       '1983-08-02T14:00:00.000000000', '1983-08-02T15:00:00.000000000',
...
       '1983-08-30T10:00:00.000000000', '1983-08-30T11:00:00.000000000',
       '1983-08-30T12:00:00.000000000', '1983-08-30T13:00:00.000000000',
       '1983-08-30T14:00:00.000000000', '1983-08-30T15:00:00.000000000',
       '1983-08-30T16:00:00.000000000', '1983-08-30T17:00:00.000000000',
       '1983-08-30T18:00:00.000000000', '1983-08-30T19:00:00.000000000',
       '1983-08-30T20:00:00.000000000', '1983-08-30T21:00:00.000000000',
       '1983-08-30T22:00:00.000000000', '1983-08-30T23:00:00.000000000',
       '1983-08-31T00:00:00.000000000', '1983-08-31T01:00:00.000000000',
       '1983-08-31T02:00:00.000000000', '1983-08-31T03:00:00.000000000',
       '1983-08-31T04:00:00.000000000', '1983-08-31T05:00:00.000000000',
       '1983-08-31T06:00:00.000000000', '1983-08-31T07:00:00.000000000',
       '1983-08-31T08:00:00.000000000', '1983-08-31T09:00:00.000000000',
       '1983-08-31T10:00:00.000000000', '1983-08-31T11:00:00.000000000',
       '1983-08-31T12:00:00.000000000', '1983-08-31T13:00:00.000000000',
       '1983-08-31T14:00:00.000000000', '1983-08-31T15:00:00.000000000',
       '1983-08-31T16:00:00.000000000', '1983-08-31T17:00:00.000000000',
       '1983-08-31T18:00:00.000000000', '1983-08-31T19:00:00.000000000',
       '1983-08-31T20:00:00.000000000', '1983-08-31T21:00:00.000000000',
       '1983-08-31T22:00:00.000000000', '1983-08-31T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 1983-08-01 ... 1983-08-31T23:00:00

In [31]:
for l in longitude:
    print( l.values )
    print( l.values%360 )
    print( '~~~~~~~~~~~~~~~~~~`' )

-180.0
180.0
~~~~~~~~~~~~~~~~~~`
-179.75
180.25
~~~~~~~~~~~~~~~~~~`
-179.5
180.5
~~~~~~~~~~~~~~~~~~`
-179.25
180.75
~~~~~~~~~~~~~~~~~~`
-179.0
181.0
~~~~~~~~~~~~~~~~~~`
-178.75
181.25
~~~~~~~~~~~~~~~~~~`
-178.5
181.5
~~~~~~~~~~~~~~~~~~`
-178.25
181.75
~~~~~~~~~~~~~~~~~~`
-178.0
182.0
~~~~~~~~~~~~~~~~~~`
-177.75
182.25
~~~~~~~~~~~~~~~~~~`
-177.5
182.5
~~~~~~~~~~~~~~~~~~`
-177.25
182.75
~~~~~~~~~~~~~~~~~~`
-177.0
183.0
~~~~~~~~~~~~~~~~~~`
-176.75
183.25
~~~~~~~~~~~~~~~~~~`
-176.5
183.5
~~~~~~~~~~~~~~~~~~`
-176.25
183.75
~~~~~~~~~~~~~~~~~~`
-176.0
184.0
~~~~~~~~~~~~~~~~~~`
-175.75
184.25
~~~~~~~~~~~~~~~~~~`
-175.5
184.5
~~~~~~~~~~~~~~~~~~`
-175.25
184.75
~~~~~~~~~~~~~~~~~~`
-175.0
185.0
~~~~~~~~~~~~~~~~~~`
-174.75
185.25
~~~~~~~~~~~~~~~~~~`
-174.5
185.5
~~~~~~~~~~~~~~~~~~`
-174.25
185.75
~~~~~~~~~~~~~~~~~~`
-174.0
186.0
~~~~~~~~~~~~~~~~~~`
-173.75
186.25
~~~~~~~~~~~~~~~~~~`
-173.5
186.5
~~~~~~~~~~~~~~~~~~`
-173.25
186.75
~~~~~~~~~~~~~~~~~~`
-173.0
187.0
~~~~~~~~~~~~~~~~~~`
-172.75
187.25
